In [1]:
# Setup Gemini
import google.generativeai as genai
from PIL import Image
import ipywidgets as widgets
import gradio as gr
from IPython.display import display, clear_output


In [2]:
# Replace this with your actual API key from Google AI Studio
genai.configure(api_key="your_key")

# Load text and vision models
text_model = genai.GenerativeModel('models/gemini-1.5-flash')
vision_model = genai.GenerativeModel(model_name="gemini-1.5-flash")


In [3]:
# Friendly greetings list
greetings = ["hi", "hello", "hey", "good morning", "good afternoon", "good evening"]

In [4]:
def creative_muse_text_feedback(user_input, history=[]):
    user_input_lower = user_input.lower()

    # Short friendly response for greetings
    if any(greet in user_input_lower for greet in greetings):
        return "Hello! 😊 How are you feeling today? Tell me what you're working on or need help with."

    # If user wants a longer prompt
    if any(keyword in user_input_lower for keyword in ["give me", "i need a prompt", "inspire me", "longer"]):
        prompt = f"""
        You are Creative Muse,  a warm AI friend helping people turn feelings into artistic ideas.

        The user says: "{user_input}"

        Respond with a detailed creative prompt or inspiration idea. Keep the tone encouraging.
        """
    else:
        # Keep it short and chill
        prompt = f"""
        You are Creative Muse, a friendly AI for artists and creators.

        The user says: "{user_input}"

        Respond briefly with encouragement and a short creative idea. Keep it 2–3 sentences max.
        """

    response = text_model.generate_content(prompt)
    return response.text

In [5]:
def creative_muse_image_feedback(image):
    """
    Open image and send to Gemini vision model with prompt for kind feedback.
    """
    prompt = (
        "You are Creative Muse, an encouraging AI muse who appreciates art. "
        "Look at this image and comment kindly on theme, technique, expression, "
        "and gently suggest ways to improve."
    )
    
    
    response = vision_model.generate_content([
        prompt,
        Image
    ])

    return response.text



In [6]:
# chat handling
def muse_chat(user_input, image_input, history=[]):
    if image_input:
        return creative_muse_image_feedback(image_input)
    elif user_input:
        return creative_muse_text_feedback(user_input)
    else:
        return "Please type something or upload an image for feedback."

In [7]:
# Gradio Interface
with gr.Blocks(title="Creative Muse") as demo:
    gr.Markdown("## 🎨 Creative Muse — Your AI Art Companion")
    gr.Markdown("Chat with Muse by typing your thoughts or uploading your art. Muse responds with encouragement, ideas, or gentle feedback.")

    with gr.Row():
        text_input = gr.Textbox(
            label="💬 Type here",
            placeholder="e.g. I feel blocked... or Hello 👋",
            lines=1,
            show_label=False
        )
        image_input = gr.Image(label="🖼️ Upload your art (optional)", type="pil")

    output = gr.Textbox(label="🪄 Muse's Response:", lines=5)

    text_input.submit(fn=muse_chat, inputs=[text_input, image_input], outputs=output)
    gr.Button("🎭 Ask Muse").click(fn=muse_chat, inputs=[text_input, image_input], outputs=output)

demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [8]:
# Test text feedback
#user_text = "I feel lost and uninspired, like all my ideas are dull."
#text_feedback = creative_muse_text_feedback(user_text)
#print("Text Feedback from Creative Muse:\n", text_feedback)

In [9]:
# Test image feedback (replace 'your_image.jpg' with your actual image file)
#image_path = r"C:\Users\HP\Pictures\Screenshots\Screenshot (4).png"

#try:
    #image_feedback = creative_muse_image_feedback(image_path)
   # print("\nImage Feedback from Creative Muse:\n", image_feedback)
#except FileNotFoundError:
    #print(f"Image file not found: {image_path}")    
